# FMCG Forecasting: Feature Engineering (Daily & Weekly)

This notebook performs both **daily** and **weekly** feature engineering for the FMCG demand forecasting project.

### Objectives:
- Load raw daily sales data (2022–2024)
- Engineer **daily-level features**:
  - Lags (`lag_1`, `lag_2`)
  - Rolling stats (mean, std)
  - Momentum, average by channel & region
- Filter the dataset for one key SKU: `MI-006`
- Save the enriched daily dataset for future use in real-time inference

---

### Weekly Aggregation (for classical ML model)
- Aggregate daily data to **weekly level**
- Extract summary statistics: average demand, price, promotion exposure, etc.
- Save weekly dataset in Delta format for modeling



## Imports

In [0]:
import sys
sys.path.append("/Workspace/Users/faron.beata@gmail.com/fmcg_forecasting/fmcg_forecasting_repo")
from utils.feature_engineering_utils import create_weekly_features
from utils.load_data import load_data
from pyspark.sql.functions import col

## Load | create features | save as delta

In [0]:

df_daily = load_data("dbfs:/FileStore/fmcg/parquet/FMCG_2022_2024.parquet", file_format="parquet")

# Create weekly features
df_weekly_final = create_weekly_features(df_daily)
df_weekly_final = df_weekly_final.filter(col("sku") == "MI-006")

# Save to Delta Table
df_weekly_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("dbfs:/FileStore/fmcg/delta/weekly_features")


df_weekly = spark.read.format("delta").load("dbfs:/FileStore/fmcg/delta/weekly_features")
df_weekly.createOrReplaceTempView("weekly_features")

## SQL check

In [0]:
%sql
-- SQL cell
SELECT * FROM weekly_features WHERE year = 2023 LIMIT 10


In [0]:
%sql
SELECT week, sku, channel, region, units_sold, target_next_week
FROM weekly_features
WHERE channel = 'Discount' AND year = 2023
ORDER BY week
